In [5]:
from mistralai import Mistral
import os
import dotenv

dotenv.load_dotenv()

api_key = os.environ["MISTRAL_API_KEY"]

client = Mistral(api_key=api_key)

file_name = "iga-01.jpg"

uploaded_pdf = client.files.upload(
    file={
        "file_name": file_name,
        "content": open(f"documents/{file_name}", "rb"),
    },
    purpose="ocr"
)  

In [7]:
client.files.retrieve(file_id=uploaded_pdf.id)

RetrieveFileOut(id='8233d360-a347-42f1-8597-aeff9ee4f616', object='file', size_bytes=441936, created_at=1741404066, filename='iga-01.jpg', purpose='ocr', sample_type='ocr_input', source='upload', deleted=False, num_lines=None)

In [8]:
signed_url = client.files.get_signed_url(file_id=uploaded_pdf.id)

In [11]:
ocr_response = client.ocr.process(
    model="mistral-ocr-latest",
    document={
        "type": "image_url",
        "image_url": signed_url.url,
    }
)

In [14]:
ocr_response.pages[0]

OCRPageObject(index=0, markdown='![img-0.jpeg](img-0.jpeg)', images=[OCRImageObject(id='img-0.jpeg', top_left_x=0, top_left_y=0, bottom_right_x=1500, bottom_right_y=1497, image_base64=None)], dimensions=OCRPageDimensions(dpi=200, height=1500, width=1500))

In [15]:
import base64
import json
from pathlib import Path

from mistralai import ImageURLChunk

image_file = Path("documents/iga-01.jpg")
assert image_file.is_file()
encoded = base64.b64encode(image_file.read_bytes()).decode()
base64_data_url = f"data:image/jpeg;base64,{encoded}"

image_response = client.ocr.process(document=ImageURLChunk(image_url=base64_data_url), model="mistral-ocr-latest")

response_dict = json.loads(image_response.json())
json_string = json.dumps(response_dict, indent=4)
print(json_string)

{
    "pages": [
        {
            "index": 0,
            "markdown": "![img-0.jpeg](img-0.jpeg)",
            "images": [
                {
                    "id": "img-0.jpeg",
                    "top_left_x": 0,
                    "top_left_y": 0,
                    "bottom_right_x": 1500,
                    "bottom_right_y": 1497,
                    "image_base64": null
                }
            ],
            "dimensions": {
                "dpi": 200,
                "height": 1500,
                "width": 1500
            }
        }
    ],
    "model": "mistral-ocr-2503-completion",
    "usage_info": {
        "pages_processed": 1,
        "doc_size_bytes": 441936
    }
}


C:\Users\USER\AppData\Local\Temp\ipykernel_46984\3506239303.py:14: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response_dict = json.loads(image_response.json())


In [24]:
import os
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

def download_iga_images(html_path=None, html_content=None, output_folder="iga_images"):
    """
    Extrait et télécharge les images du circulaire IGA.
    
    Args:
        html_path (str): Chemin vers le fichier HTML (optionnel)
        html_content (str): Contenu HTML (optionnel, alternativement à html_path)
        output_folder (str): Dossier de destination pour les images téléchargées
    """
    # Vérification des arguments
    if html_path is None and html_content is None:
        print("Erreur: Veuillez fournir soit le chemin du fichier HTML, soit son contenu.")
        return
    
    # Lecture du contenu HTML depuis le fichier si spécifié
    if html_path is not None:
        try:
            with open(html_path, 'r', encoding='utf-8') as f:
                html_content = f.read()
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier HTML: {str(e)}")
            return
    
    # Création du dossier de sortie s'il n'existe pas
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Dossier '{output_folder}' créé.")
    
    # Analyse du HTML avec BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Trouver toutes les balises d'image du circulaire
    # Dans le HTML fourni, les images du circulaire ont un pattern spécifique
    images = soup.find_all('img', src=re.compile(r'./circulaire/iga-\d+\.jpg'))
    
    if not images:
        print("Aucune image de circulaire trouvée dans le HTML.")
        return
    
    # Extraction de la base URL pour générer les URL complètes
    base_url = "https://www.circulaires.com/supermarche-iga/"
    
    # Téléchargement de chaque image
    downloaded_count = 0
    for idx, img in enumerate(images, start=1):
        # Récupération de l'URL relative de l'image
        img_url_relative = img.get('src')
        
        # Nettoyage de l'URL relative (suppression des paramètres éventuels)
        img_url_relative = img_url_relative.split('?')[0]
        
        # Conversion de l'URL relative en URL absolue
        img_url = urljoin(base_url, img_url_relative)

        print(img_url)
        
        # Extraction du nom de fichier de l'image
        img_filename = os.path.basename(img_url_relative)
        img_path = os.path.join(output_folder, img_filename)
        
        try:
            # Téléchargement de l'image
            response = requests.get(img_url, stream=True)
            response.raise_for_status()
            
            # Sauvegarde de l'image sur le disque
            with open(img_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
            
            print(f"[{idx}/{len(images)}] Image téléchargée: {img_filename}")
            downloaded_count += 1
        
        except Exception as e:
            print(f"Erreur lors du téléchargement de {img_filename}: {str(e)}")
    
    print(f"\nTéléchargement terminé. {downloaded_count}/{len(images)} images sauvegardées dans '{output_folder}'.")



In [25]:

download_iga_images(html_path="page.html")

https://www.circulaires.com/supermarche-iga/circulaire/iga-01.jpg
[1/26] Image téléchargée: iga-01.jpg
https://www.circulaires.com/supermarche-iga/circulaire/iga-02.jpg
[2/26] Image téléchargée: iga-02.jpg
https://www.circulaires.com/supermarche-iga/circulaire/iga-03.jpg
[3/26] Image téléchargée: iga-03.jpg
https://www.circulaires.com/supermarche-iga/circulaire/iga-04.jpg
[4/26] Image téléchargée: iga-04.jpg
https://www.circulaires.com/supermarche-iga/circulaire/iga-05.jpg
[5/26] Image téléchargée: iga-05.jpg
https://www.circulaires.com/supermarche-iga/circulaire/iga-06.jpg
[6/26] Image téléchargée: iga-06.jpg
https://www.circulaires.com/supermarche-iga/circulaire/iga-07.jpg
[7/26] Image téléchargée: iga-07.jpg
https://www.circulaires.com/supermarche-iga/circulaire/iga-08.jpg
[8/26] Image téléchargée: iga-08.jpg
https://www.circulaires.com/supermarche-iga/circulaire/iga-09.jpg
[9/26] Image téléchargée: iga-09.jpg
https://www.circulaires.com/supermarche-iga/circulaire/iga-10.jpg
[10/26]